# =============================================================================
# NOTEBOOK 04 - TESTS ET ÉVALUATION
# =============================================================================


## 🧪 Tests et Évaluation du Modèle
# 
# Ce notebook couvre:
# 1. Évaluation sur le jeu de test
# 2. Matrice de confusion
# 3. Métriques par classe
# 4. Analyse des erreurs
# 5. Tests sur images réelles

In [ ]:
# Imports
!pip install -q tensorflow scikit-learn seaborn

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support
)
from pathlib import Path
import json
from tqdm.notebook import tqdm
import cv2

In [ ]:
# Charger le modèle et les données
from google.colab import drive
drive.mount('/content/drive')

MODEL_DIR = Path('/content/drive/MyDrive/drone-agri-ai/models')
DATA_DIR = Path('/content/drive/MyDrive/drone-agri-ai/data')
TEST_DIR = DATA_DIR / 'test'

# Charger le modèle
model = keras.models.load_model(MODEL_DIR / 'plant_model.keras')

# Charger le mapping des classes
with open(MODEL_DIR / 'class_mapping.json', 'r') as f:
    class_mapping = json.load(f)

class_names = class_mapping['class_names']
num_classes = len(class_names)
print(f"Classes: {num_classes}")

In [ ]:
# Préparer le générateur de test
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda x: (x - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

print(f"Images de test: {test_generator.samples}")

In [ ]:
# Prédictions sur le jeu de test
print("🔮 Génération des prédictions...")

# Pour le modèle multi-sorties, on se concentre sur la classification
predictions = model.predict(test_generator, verbose=1)

# Extraire les prédictions de classification
if isinstance(predictions, dict):
    class_predictions = predictions['classification']
else:
    class_predictions = predictions[1]  # Index de la sortie classification

y_pred = np.argmax(class_predictions, axis=1)
y_true = test_generator.classes

print(f"Prédictions: {y_pred.shape}")
print(f"Labels: {y_true.shape}")

# Accuracy globale
accuracy = accuracy_score(y_true, y_pred)
print(f"\n📊 ACCURACY GLOBALE: {accuracy * 100:.2f}%")

# Rapport de classification
print("\n📋 RAPPORT DE CLASSIFICATION")
print("="*60)

report = classification_report(
    y_true, 
    y_pred, 
    target_names=class_names,
    output_dict=True
)

# Afficher les métriques principales
print(f"\nAccuracy: {report['accuracy']:.4f}")
print(f"Macro Avg Precision: {report['macro avg']['precision']:.4f}")
print(f"Macro Avg Recall: {report['macro avg']['recall']:.4f}")
print(f"Macro Avg F1-Score: {report['macro avg']['f1-score']:.4f}")

In [ ]:
# Top 10 meilleures et pires classes
class_metrics = []
for class_name in class_names:
    if class_name in report:
        class_metrics.append({
            'class': class_name,
            'precision': report[class_name]['precision'],
            'recall': report[class_name]['recall'],
            'f1': report[class_name]['f1-score'],
            'support': report[class_name]['support']
        })

df_metrics = pd.DataFrame(class_metrics)
df_metrics = df_metrics.sort_values('f1', ascending=False)

print("\n🏆 TOP 10 MEILLEURES CLASSES:")
print(df_metrics.head(10).to_string(index=False))

print("\n⚠️ TOP 10 PIRES CLASSES:")
print(df_metrics.tail(10).to_string(index=False))

In [ ]:
# Matrice de confusion (version réduite)
# Pour 38 classes, la matrice complète est trop grande
# On va créer une version agrégée par plante

def get_plant_name(class_name):
    """Extrait le nom de la plante"""
    parts = class_name.split('___')
    return parts[0] if len(parts) >= 1 else class_name

# Mapper les classes aux plantes
plant_mapping = {i: get_plant_name(name) for i, name in enumerate(class_names)}
plants = sorted(set(plant_mapping.values()))
plant_to_idx = {p: i for i, p in enumerate(plants)}

# Convertir les labels en index de plantes
y_true_plants = [plant_to_idx[plant_mapping[y]] for y in y_true]
y_pred_plants = [plant_to_idx[plant_mapping[y]] for y in y_pred]

# Matrice de confusion par plante
cm_plants = confusion_matrix(y_true_plants, y_pred_plants)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm_plants, 
    annot=True, 
    fmt='d',
    xticklabels=plants,
    yticklabels=plants,
    cmap='Blues'
)
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de confusion par type de plante')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(MODEL_DIR / 'confusion_matrix_plants.png', dpi=150)
plt.show()

# Analyser les erreurs fréquentes
print("\n🔍 ANALYSE DES ERREURS FRÉQUENTES")
print("="*60)

errors = []
for i in range(len(y_true)):
    if y_true[i] != y_pred[i]:
        errors.append({
            'true': class_names[y_true[i]],
            'pred': class_names[y_pred[i]],
            'true_plant': get_plant_name(class_names[y_true[i]]),
            'pred_plant': get_plant_name(class_names[y_pred[i]])
        })

df_errors = pd.DataFrame(errors)

# Erreurs les plus fréquentes
error_counts = df_errors.groupby(['true', 'pred']).size().reset_index(name='count')
error_counts = error_counts.sort_values('count', ascending=False)

print("\nTop 15 confusions les plus fréquentes:")
for _, row in error_counts.head(15).iterrows():
    print(f"  {row['true'][:30]:30} -> {row['pred'][:30]:30} ({row['count']}x)")

# Visualiser les erreurs
print("\n📸 VISUALISATION DES ERREURS")

# Trouver des exemples d'erreurs
error_indices = np.where(y_true != y_pred)[0]

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, idx in enumerate(error_indices[:12]):
    # Charger l'image
    img_path = test_generator.filepaths[idx]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ax = axes[i]
    ax.imshow(img)
    ax.set_title(f"Réel: {class_names[y_true[idx]][:20]}\nPrédit: {class_names[y_pred[idx]][:20]}", fontsize=8)
    ax.axis('off')

plt.tight_layout()
plt.savefig(MODEL_DIR / 'error_examples.png', dpi=150)
plt.show()

In [ ]:
# Tester le modèle TFLite
print("\n🔧 TEST DU MODÈLE TFLITE")
print("="*60)

tflite_path = MODEL_DIR / 'plant_model.tflite'
interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input: {input_details[0]['shape']}, {input_details[0]['dtype']}")
print(f"Outputs: {len(output_details)}")

# Tester sur quelques images
def predict_tflite(image_path):
    """Prédit avec le modèle TFLite"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32) / 255.0
    
    # Normalisation
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (img - mean) / std
    
    img = np.expand_dims(img, axis=0).astype(np.float32)
    
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    
    # Récupérer les sorties
    outputs = {}
    for detail in output_details:
        name = detail['name']
        tensor = interpreter.get_tensor(detail['index'])
        outputs[name] = tensor
    
    return outputs

# Tester
sample_images = list(TEST_DIR.rglob('*.jpg'))[:5]
for img_path in sample_images:
    outputs = predict_tflite(str(img_path))
    
    # Trouver la sortie de classification
    for name, tensor in outputs.items():
        if 'classification' in name.lower() or tensor.shape[-1] == num_classes:
            pred_idx = np.argmax(tensor[0])
            confidence = np.max(tensor[0])
            print(f"{img_path.parent.name} -> {class_names[pred_idx]} ({confidence*100:.1f}%)")
            break

In [ ]:
# Rapport final
print("\n" + "="*60)
print("📊 RAPPORT FINAL D'ÉVALUATION")
print("="*60)

summary = {
    "accuracy": float(accuracy),
    "num_classes": num_classes,
    "test_samples": len(y_true),
    "macro_precision": float(report['macro avg']['precision']),
    "macro_recall": float(report['macro avg']['recall']),
    "macro_f1": float(report['macro avg']['f1-score']),
    "num_errors": len(errors),
    "error_rate": len(errors) / len(y_true),
    "best_classes": df_metrics.head(5)['class'].tolist(),
    "worst_classes": df_metrics.tail(5)['class'].tolist()
}

print(f"""
Accuracy globale:     {summary['accuracy']*100:.2f}%
Classes:              {summary['num_classes']}
Échantillons test:    {summary['test_samples']}

Macro Precision:      {summary['macro_precision']:.4f}
Macro Recall:         {summary['macro_recall']:.4f}
Macro F1-Score:       {summary['macro_f1']:.4f}

Erreurs:              {summary['num_errors']} ({summary['error_rate']*100:.2f}%)

Meilleures classes:   {', '.join(summary['best_classes'][:3])}
Pires classes:        {', '.join(summary['worst_classes'][:3])}
""")

# Sauvegarder le rapport
with open(MODEL_DIR / 'evaluation_report.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Rapport sauvegardé!")